# 1. Google Selenium

* 웹 동작, 코드 가져오기에 쓰임.
* 동적 크롤링에 효과적

In [3]:
import requests
import pandas as pd

In [4]:
# 이런 식으로는 google에서 html 코드를 불러올 수 없음
url = 'https://google.com'
req = requests.get(url)
html = req.text
#html

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [6]:
driver = webdriver.Chrome('/Workspace/02. DataAnalysis/05.Crawling/chromedriver')
driver.get(url)

In [7]:
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [9]:
divs = driver.find_elements_by_css_selector('#search .g')
len(divs)

10

In [11]:
div = divs[0]
title = div.find_element_by_css_selector('.LC20lb.DKV0Md').text
title

'ChromeDriver - WebDriver for Chrome - Downloads'

In [12]:
content = div.find_element_by_css_selector('.VwiC3b.MUxGbd').text
content

'Current Releases · ChromeDriver 93.0.4577.15 · Supports Chrome version · For more details, please see the · ChromeDriver 92.0.4515.107 · Supports Chrome version 92.'

In [13]:
title_list, content_list = [], []
for div in divs :
    title = div.find_element_by_css_selector('.LC20lb.DKV0Md').text
    content = div.find_element_by_css_selector('.VwiC3b.MUxGbd').text
    title_list.append(title)
    content_list.append(content)

In [14]:
df = pd.DataFrame({
    '제목': title_list, '내용': content_list
})
df

,제목,내용
0,ChromeDriver - WebDriver for Chrome - Downloads,Current Releases · ChromeDriver 93.0.4577.15 ·...
1,ChromeDriver - WebDriver for Chrome - Downloads,Current Releases · ChromeDriver 93.0.4577.15 ·...
2,ChromeDriver - WebDriver for Chrome - Google S...,It provides capabilities for navigating to web...
3,selenium) 설치하기! chromedriver 버전 오류 해결하기 - 네이버 ...,여기에서 현재 사용중인 크롬의 버전과 일치하는 버전의 파일을 다운 받으시면 됩니다....
4,[ChromeDriver] 크롬 드라이버 버전에 따라 설정하는 방법,2020. 6. 9. — 크롬드라이버 다운로드 사이트로 갑니다. https://ch...
5,크롬드라이버(chromedriver) 다운로드 - Mizy's - 티스토리,크롬드라이버(chromedriver) 다운로드. Python/Crawling 202...
6,[Python]크롬 드라이버 버전 오류 SessionNotCreatedException,"Chrome(executable_path=""C:/python/chromedriver..."
7,ChromeDriver - GitHub,이 페이지에 관한 정보가 없습니다.
8,Selenium을 사용하고 싶습니다 - goorm,ChromeDriver 다운로드. 아래 명령을 사용하여 크롬 드라이버를 다운로드 받...
9,selenium 버전 문제 해결 - This version of ChromeDriv...,2019. 11. 30. — This version of ChromeDriver o...


In [15]:
driver.close()

# 2. Youtube Selenium and Beautiful Soup

In [16]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [18]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--diable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [19]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
time.sleep(2)

In [20]:
trs = driver.find_elements_by_css_selector('.aos-int')
len(trs)

0

In [21]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')   

In [27]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [28]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [29]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [30]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text
subscriber, view, video

('6400만', '190억0381만', '371개')

In [32]:
channels = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[-1]
    channels.append([category, name, subscriber, view, video])

In [34]:
df = pd.DataFrame(channels, columns = ['카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,개
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6621만,개
97,음악/댄스/가수,NCT DREAM,387만,3억7615만,개
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,개
99,취미/라이프,JaeYeol ASMR 재열,380만,10억1154만,개


In [36]:
def convert_unit(s):
    s = s.replace('억','').replace('개','').replace(',','')
    s = s.replace('만','0000')
    return f'{int(s):,d}'

In [37]:
# 두번째 페이지 - xpath로 찾아 click()하여 이동하기
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [38]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [39]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [42]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [43]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,애완/반려동물,크림히어로즈,"3,800,000","1,374,770,000",940
1,BJ/인물/연예인,워크맨-Workman,"3,780,000","679,370,000",141
2,BJ/인물/연예인,waveya 2011,"3,730,000","1,346,280,000",702
3,키즈/어린이,Lime Tube[라임튜브],"3,730,000","2,595,420,000","1,723"
4,음악/댄스/가수,WINNER,"3,730,000","1,171,260,000",428


In [44]:
# 페이지 1에서 10까지 크롤링 하기
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [48]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,미분류,밉지않은 관종언니,"510,000","67,950,000",170
996,게임,미소,"510,000","210,780,000","3,084"
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


In [46]:
df.to_csv('유튜브_순위.csv', index=None)

In [47]:
driver.close()

# 3. Youtube Selenium

In [49]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [50]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 

In [51]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [52]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [53]:
# pop-up 창이 있으면, pop-up 창을 닫고 메인 윈도우로 되돌아 옴
window_len = len(driver.window_handles)
if window_len == 2:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()

# 화면의 끝까지 PageDown으로 내려보냄
driver.switch_to.window(driver.window_handles[0])
body = driver.find_element_by_css_selector('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [54]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [55]:
trs[99].text

'101\n[취미/라이프]\nJaeYeol ASMR 재열 0\n먹방 asmr mukbang "jaeyeol asmr" 재열 MUKBANG "ASMR MUKBANG"\n380만 10억1154만 970개 3,100'

In [57]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [58]:
tr = trs[99]
category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
category

'취미/라이프'

In [59]:
name = tr.find_element_by_css_selector('.subject a').text.strip()
subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
name, subscriber, view, video

('JaeYeol ASMR 재열', '3,800,000', '1,011,540,000', '970')

In [60]:
from tqdm.notebook import tqdm

In [61]:
results = []
for page in tqdm(range(1,11)):
    #print(page)
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)

    window_len = len(driver.window_handles)
    if window_len == 2:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    trs = driver.find_elements_by_css_selector('.aos-init')

    for tr in trs:
        category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
        #category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = tr.find_element_by_css_selector('.subject a').text.strip()
        #name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
        #subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
        #view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
        #video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [62]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,735,840,000",654
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,895,380,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [63]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,미분류,밉지않은 관종언니,"510,000","67,950,000",170
996,게임,미소,"510,000","210,780,000","3,084"
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"
